In [ ]:
# 必要なライブラリのインストールとインポート
!pip install compressai

import torch
import matplotlib.pyplot as plt
from compressai.zoo import mbt2018

print("✅ セットアップ完了！")


In [ ]:
# 訓練済みモデルの読み込み
device = "cuda" if torch.cuda.is_available() else "cpu"
model = mbt2018(quality=3, pretrained=True, progress=True)
model = model.to(device)
model.eval()

print(f"✅ 訓練済みモデル読み込み完了 (デバイス: {device})")


In [ ]:
# シンプルなテスト画像を作成
test_image = torch.zeros(1, 3, 256, 256).to(device)
test_image[0, 0, :128, :128] = 1.0  # 左上: 赤
test_image[0, 1, :128, 128:] = 1.0  # 右上: 緑
test_image[0, 2, 128:, :128] = 1.0  # 左下: 青
test_image[0, :, 128:, 128:] = 0.5  # 右下: グレー

print("✅ テスト画像作成完了")


In [ ]:
# 画像圧縮・展開の実行
with torch.no_grad():
    # 圧縮
    compressed = model.compress(test_image)
    # 展開
    decompressed = model.decompress(compressed["strings"], compressed["shape"])
    reconstructed = decompressed["x_hat"]

print("✅ 圧縮・展開完了")


In [ ]:
# 🎯 最終結果の表示
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 元画像
original_img = test_image[0].cpu().permute(1, 2, 0).clamp(0, 1)
axes[0].imshow(original_img)
axes[0].set_title('元画像', fontsize=16, fontweight='bold')
axes[0].axis('off')

# 圧縮・復元後の画像
reconstructed_img = reconstructed[0].cpu().permute(1, 2, 0).clamp(0, 1)
axes[1].imshow(reconstructed_img)
axes[1].set_title('圧縮・復元後', fontsize=16, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

# 圧縮効果の数値
original_size = test_image.numel() * 8  # bits
compressed_size = len(compressed['strings'][0]) * 8  # bits
compression_ratio = original_size / compressed_size
mse = torch.mean((test_image - reconstructed) ** 2).item()
psnr = -10 * torch.log10(torch.tensor(mse))

print("\n🎯 === 最終結果 ===")
print(f"📊 圧縮率: {compression_ratio:.1f}倍")
print(f"📈 PSNR: {psnr:.1f} dB")
print(f"💾 元サイズ: {original_size//8:,} bytes")
print(f"🗜️ 圧縮後: {len(compressed['strings'][0]):,} bytes")
print("\n✨ 学習ベース画像圧縮の成功！")
